In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

for computer vision fastai.vision

for text fastai.text

In [3]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/home/jupyter/.fastai/data/imdb_sample/texts.csv')]

In [4]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


Reviews -- each review classified as a negative or positive -- label



In [5]:
df['text'][1]

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [6]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [7]:
data_lm.save()

In [8]:
data = load_data(path)

## Tokenization

Each thing with spaces represent a single linguistic concept -- find words that are very rare and stores it as special token

Token of words

## Numericalization

Once we have extracted tokens from our texts, we convert to integers by creating a list of all the words used. We only keep the ones that appear at least twice with a maximum vocabulary size of 60,000 (by default) and replace the ones that don't make the cut by the unknown token `UNK`.

In [9]:
data.vocab.itos[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']

vocab gives the list of all the unique tokens -- this gives the first 10 items 

Replace every movie review with a list of numbers-- each token has a number

In [10]:
data.train_ds[0][0]

Text xxbos xxmaj most italian horror lovers seem to hate this movie since because it has no connection to the first two xxmaj demons films . xxmaj and with the " xxmaj demons xxup iii " in the title , one would assume it would . xxmaj the problem is that this film was never intended to be part of the xxmaj demons series . xxmaj the distributors only a " xxmaj demons xxup iii " above its original title " xxmaj the xxmaj ogre " to cash in on the other films popularity . xxmaj the new xxmaj american xxup dvd release of this picture has the title " xxmaj demons xxup iii : xxmaj the xxmaj ogre " on the box art but the film itself only says " xxmaj the xxmaj ogre " . i do n't know if past releases had the title " xxmaj demons xxup iii " on the actual film itself , but this new release just seems to be a little white lie . xxmaj if you can get past the " xxmaj demons xxup iii " in the title , you might some enjoyment in " xxmaj the xxmaj ogre " . xxmaj it starts out with a creep intro , and s

In [11]:
data.train_ds[0][0].data[:10]

array([   2,    5,  109, 1705,  219, 1574,  326,   15,  984,   21])

Datablock api does everything at once -- tokenization, Numericalization and creating a databunch

In [12]:
data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

## Model

In NLP we have 2 models

1. Language model 
2. Classifier

In [13]:
bs=48